<h3><i>Data Pipeline</i> </h3>

In [52]:
import os
import pandas as pd

In [53]:
os.chdir(r'E:\Python\etl\fastapi etl pipeline\Data')
cwd=os.getcwd()

<h4><i>Data Load</i></h4>

In [54]:
for f in os.listdir(cwd):
    print(f)

2018.csv
2019.csv
2020.csv
cache


In [55]:
df=pd.read_csv('2018.csv')
df.head(5)

,Rank,Release Group,Worldwide,Domestic,%,Foreign,%.1
0,1,Avengers: Infinity War,"$2,048,359,754","$678,815,482",33.1%,"$1,369,544,272",66.9%
1,2,Black Panther,"$1,346,913,161","$700,059,566",52%,"$646,853,595",48%
2,3,Jurassic World: Fallen Kingdom,"$1,308,467,944","$417,719,760",31.9%,"$890,748,184",68.1%
3,4,Incredibles 2,"$1,242,805,359","$608,581,744",49%,"$634,223,615",51%
4,5,Aquaman,"$1,148,461,807","$335,061,807",29.2%,"$813,400,000",70.8%


<h4><i>Data Merge</i></h4>

In [56]:

dataframe=[]
csv_files=[x for x in os.listdir(cwd) if x.endswith('.csv')]
for f in csv_files:
    print(f)
    year=f.replace('.csv','')
    csv=os.path.join(os.getcwd(),f)
    this_df=pd.read_csv(csv)
    this_df['filename']=f
    this_df['year']=year
    print(this_df.head(1))
    dataframe.append(this_df)

2018.csv
   Rank           Release Group       Worldwide      Domestic      %  \
0     1  Avengers: Infinity War  $2,048,359,754  $678,815,482  33.1%   

          Foreign    %.1  filename  year  
0  $1,369,544,272  66.9%  2018.csv  2018  
2019.csv
   Rank      Release Group       Worldwide      Domestic      %  \
0     1  Avengers: Endgame  $2,797,800,564  $858,373,000  30.7%   

          Foreign    %.1  filename  year  
0  $1,939,427,564  69.3%  2019.csv  2019  
2020.csv
   Rank      Release Group     Worldwide      Domestic    %       Foreign  \
0     1  Bad Boys for Life  $425,514,702  $204,417,855  48%  $221,096,847   

   %.1  filename  year  
0  52%  2020.csv  2020  


In [57]:
tot_df=pd.concat(dataframe)
tot_df.tail(2)

,Rank,Release Group,Worldwide,Domestic,%,Foreign,%.1,filename,year
30,31,Run,"$4,052",-,-,"$4,052",100%,2020.csv,2020
31,32,A Quiet Place Part II,$163,-,-,$163,100%,2020.csv,2020


In [58]:
tot_df.to_csv(r'cache\movies_data.csv',index=False)

<h4><i>Data Clean</i></h4>

In [59]:
fdf=pd.read_csv(r'cache\movies_data.csv')

In [60]:
fdf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1616 entries, 0 to 1615
Data columns (total 9 columns):
Rank             1616 non-null int64
Release Group    1616 non-null object
Worldwide        1616 non-null object
Domestic         1616 non-null object
%                1616 non-null object
Foreign          1616 non-null object
%.1              1616 non-null object
filename         1616 non-null object
year             1616 non-null int64
dtypes: int64(2), object(7)
memory usage: 113.7+ KB


In [61]:
fdf['Rank']=-1

In [62]:
fdf['Domestic %']=fdf['%']
fdf['Foreign %']=fdf['%.1']

In [64]:
fdf.drop(['%','%.1'],axis=1,inplace=True)

In [65]:
fdf.head()

,Rank,Release Group,Worldwide,Domestic,Foreign,filename,year,Domestic %,Foreign %
0,-1,Avengers: Infinity War,"$2,048,359,754","$678,815,482","$1,369,544,272",2018.csv,2018,33.1%,66.9%
1,-1,Black Panther,"$1,346,913,161","$700,059,566","$646,853,595",2018.csv,2018,52%,48%
2,-1,Jurassic World: Fallen Kingdom,"$1,308,467,944","$417,719,760","$890,748,184",2018.csv,2018,31.9%,68.1%
3,-1,Incredibles 2,"$1,242,805,359","$608,581,744","$634,223,615",2018.csv,2018,49%,51%
4,-1,Aquaman,"$1,148,461,807","$335,061,807","$813,400,000",2018.csv,2018,29.2%,70.8%


In [72]:
clean_col=['Worldwide','Domestic','Foreign']

def cur_to_int(val):
    val=val.replace('$',"").replace(',',"")
    try:
        val=int(val)
    except:
#         If no data is avialable 
        val=0
    return val

def clean_data(row):
    for i in clean_col:
        val=row[i]
        row[i]=cur_to_int(val)
    return row
clean_df=fdf.apply(clean_data,axis=1)
clean_df.head(1)

,Rank,Release Group,Worldwide,Domestic,Foreign,filename,year,Domestic %,Foreign %
0,-1,Avengers: Infinity War,2048359754,678815482,1369544272,2018.csv,2018,33.1%,66.9%


In [74]:
clean_df.dtypes

Rank              int64
Release Group    object
Worldwide         int64
Domestic          int64
Foreign           int64
filename         object
year              int64
Domestic %       object
Foreign %        object
dtype: object

In [76]:
clean_df.sort_values(by=['Worldwide'],inplace=True,ascending=False)
clean_df.head()

,Rank,Release Group,Worldwide,Domestic,Foreign,filename,year,Domestic %,Foreign %
899,-1,Avengers: Endgame,2797800564,858373000,1939427564,2019.csv,2019,30.7%,69.3%
0,-1,Avengers: Infinity War,2048359754,678815482,1369544272,2018.csv,2018,33.1%,66.9%
900,-1,The Lion King,1656943394,543638043,1113305351,2019.csv,2019,32.8%,67.2%
901,-1,Frozen II,1450026933,477373578,972653355,2019.csv,2019,32.9%,67.1%
1,-1,Black Panther,1346913161,700059566,646853595,2018.csv,2018,52%,48%


In [79]:
# clean_df.reset_index(inplace=True,drop=True)
clean_df.head(4)

,Rank,Release Group,Worldwide,Domestic,Foreign,filename,year,Domestic %,Foreign %
0,-1,Avengers: Endgame,2797800564,858373000,1939427564,2019.csv,2019,30.7%,69.3%
1,-1,Avengers: Infinity War,2048359754,678815482,1369544272,2018.csv,2018,33.1%,66.9%
2,-1,The Lion King,1656943394,543638043,1113305351,2019.csv,2019,32.8%,67.2%
3,-1,Frozen II,1450026933,477373578,972653355,2019.csv,2019,32.9%,67.1%


In [81]:
# clean_df['Rank']=clean_df.index+1
clean_df.tail()

,Rank,Release Group,Worldwide,Domestic,Foreign,filename,year,Domestic %,Foreign %
1611,1612,TVTV: Video Revolutionaries,812,812,0,2018.csv,2018,100%,-
1612,1613,The Untold Story,790,790,0,2019.csv,2019,100%,-
1613,1614,The Portal,685,0,685,2019.csv,2019,-,100%
1614,1615,Higher Power,528,528,0,2018.csv,2018,100%,-
1615,1616,A Quiet Place Part II,163,0,163,2020.csv,2020,-,100%


In [84]:
clean_df.to_csv(r'cache\final_movies.csv',index=False)

<h4><i>Data as API</i></h4>